In [2]:
!pip install vectorbt

  Created wheel for vectorbt: filename=vectorbt-0.21.0-py3-none-any.whl size=535704 sha256=d7212c069d8632bab74716621d8a6def7f41a90d768a6c726befca26af6063c9
  Stored in directory: c:\users\btk\appdata\local\pip\cache\wheels\b0\6d\b8\d28e8ccea5d97b9b1439e5d7f848a85bc72c7b8151b06fc628
Successfully built vectorbt


In [3]:
import pandas as pd
import numpy as np
import vectorbt as vbt

In [6]:
columns = {
    'Date': 'date',
    'Tipo': 'op_type',
    'Volume': 'contracts',
    'Price': 'entry_price',
    'SL': 'stop_loss',
    'TP': 'take_profit',
    'Saldo': 'capital',
    'Price Vol': 'price_change_volatility',
    'Scaled Price Return': 'scaled_price_change_volatility',
    'Retorno': 'capital_return',
    'Dummy': 'gain_loss'
}

df = pd.read_csv('C:/Users/BTK/Desktop/Git/backtest-model-ai/database/polentovisk_2018_2021.csv', sep=';')
df.rename(columns=columns, inplace=True)
df

,date,op_type,contracts,entry_price,stop_loss,take_profit,capital,price_change_volatility,scaled_price_change_volatility,capital_return,gain_loss
0,2018.01.02 16:57:00,sell,19,78310,77440,80340,1002206,"0,0001","-0,028285311",2206,1
1,2018.01.02 17:00:01,sell,19,78310,77440,80340,1004412,"0,0000","-0,032808644",2206,1
2,2018.01.04 16:57:00,sell,19,79045,78730,81630,1004499,"0,0094","0,454192378",87,1
3,2018.01.04 17:00:01,sell,19,79160,78730,81630,1005023,"0,0015","0,042680273",524,1
4,2018.01.08 11:34:21,buy,38,79400,79390,76490,1002747,"0,0030","0,124504574",-2276,0
...,...,...,...,...,...,...,...,...,...,...,...
741,2021.10.13 16:50:14,sell,38,114605,114205,117105,1367659,"0,0165","0,820855657",860,1
742,2021.10.15 16:57:00,sell,19,115740,114935,117835,1369618,"0,0099","0,481059923",1959,1
743,2021.10.15 17:00:01,sell,19,115675,114935,117835,1371330,"-0,0006","-0,061948649",1712,1
744,2021.10.28 11:23:01,sell,38,107475,107475,110375,1369130,"-0,0709","-3,710998149",-2200,0


In [7]:
for col in df.columns:
    if col == 'price_change_volatility' or col == 'scaled_price_change_volatility':
        for row in range(len(df[col])):
            string = str(df[col][row])
            df[col][row] = float(string.replace(',', '.'))

<ipython-input-7-3df9f031eb16>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][row] = float(string.replace(',', '.'))


In [9]:
df['date'] = pd.to_datetime(df['date'])

In [10]:
df_win_2019 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2019_BMF_T.csv')
df_win_2020 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2020_BMF_T.csv')
df_win_2021 = pd.read_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2021_BMF_T.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/OHLC_1SWIN$N_2019_BMF_T.csv'

In [210]:
df_win_2019.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)
df_win_2020.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)
df_win_2021.rename(columns={'<DATE>_<TIME>': 'date'}, inplace=True)

df_win_2019['date'] = pd.to_datetime(df_win_2019['date'])
df_win_2020['date'] = pd.to_datetime(df_win_2020['date'])
df_win_2021['date'] = pd.to_datetime(df_win_2021['date'])

In [211]:
df_win_2019.set_index('date', drop=True, inplace=True)
df_win_2020.set_index('date', drop=True, inplace=True)
df_win_2021.set_index('date', drop=True, inplace=True)

df.set_index('date', drop=True, inplace=True)

merge_df = pd.concat([df_win_2019, df_win_2020, df_win_2021])

In [213]:
columns_win = {
    '<OPEN>': 'open', 
    '<HIGH>': 'high', 
    '<LOW>': 'low', 
    '<CLOSE>': 'close', 
    '<TICK>': 'tick', 
    '<VOL>': 'volume'
}

merge_df.rename(columns=columns_win, inplace=True)

In [216]:
returns = [0]

for i in range(1, len(merge_df['close'])):
    returns.append((merge_df['close'][i] - merge_df['close'][i-1])/merge_df['close'][i-1])
returns

merge_df['log_returns'] = returns

In [217]:
scaled_returns = []

mean = merge_df['log_returns'].mean()
std = merge_df['log_returns'].std()

for i in range(len(merge_df['log_returns'])):
    scaled_returns.append((merge_df['log_returns'][0] - mean)/std)
    
merge_df['scaled_log_return'] = scaled_returns

In [218]:
from math import pi, sqrt, exp

proba_return = []

for i in range(len(merge_df['scaled_log_return'])):
    proba_return.append(1/(sqrt(2*pi)*exp(-0.5*merge_df['scaled_log_return'][i]*merge_df['scaled_log_return'][i])))
    
merge_df['probability_chance'] = proba_return

In [11]:
mean_candle = []

for i in range(len(merge_df['scaled_log_return'])):
    mean_candle.append(merge_df['high'][i] - merge_df['low'][i])
    
merge_df['candle_size'] = mean_candle

NameError: name 'merge_df' is not defined

In [220]:
merge_df['M20'] = vbt.MA.run(merge_df['candle_size'], 20).ma
merge_df['M10'] = vbt.MA.run(merge_df['candle_size'], 10).ma
merge_df['M5'] = vbt.MA.run(merge_df['candle_size'], 5).ma
merge_df['M2'] = vbt.MA.run(merge_df['candle_size'], 2).ma

merge_df['std_5S'] = merge_df['log_returns'].rolling(window=5).std()
merge_df['std_30S'] = merge_df['log_returns'].rolling(window=30).std()
merge_df['std_60S'] = merge_df['log_returns'].rolling(window=60).std()
merge_df['std_5M'] = merge_df['log_returns'].rolling(window=300).std()
merge_df['std_10M'] = merge_df['log_returns'].rolling(window=600).std()
merge_df['std_30M'] = merge_df['log_returns'].rolling(window=1800).std()
merge_df['std_1H'] = merge_df['log_returns'].rolling(window=3600).std()
merge_df['std_2H'] = merge_df['log_returns'].rolling(window=7200).std()

In [1]:
merge_df['M20'].fillna(merge_df['M20'].median(), inplace=True)
merge_df['M10'].fillna(merge_df['M10'].median(), inplace=True)
merge_df['M5'].fillna(merge_df['M5'].median(), inplace=True)
merge_df['M2'].fillna(merge_df['M2'].median(), inplace=True)

merge_df['std_5S'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_30S'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_60S'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_5M'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_10M'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_30M'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_1H'].fillna( merge_df['log_returns'].median(), inplace=True)
merge_df['std_2H'].fillna( merge_df['log_returns'].median(), inplace=True)


NameError: name 'merge_df' is not defined

In [222]:
to_drop = ['Open', 'High', 'Low', 'Close']

merge_df.drop(to_drop, axis=1, inplace=True)

,Tipo,Volume,Price,SL,TP,Saldo,Price Vol,Scaled Price Return,Retorno,Dummy,...,Tick,Vol,Returns,Scaled Returns,Proba Returns,Candle Size,MM20,MM10,MM5,MM2
date,,,,,,,,,,,,,,,,,,,,,
2019-04-02 15:30:14,buy,38,95640,96035,93135,1093457,0.0059,0.27554,862,1,...,141,710,0.000000,-0.043876,0.399326,25.0,12.00,11.5,12.0,10.0
2019-04-02 15:53:14,sell,38,95315,95340,98240,1091047,-0.0034,-0.209129,-2410,0,...,32,108,-0.001046,-0.043876,0.399326,5.0,12.00,11.5,12.0,15.0
2019-04-03 14:43:08,sell,38,96075,96075,98975,1088827,0.008,0.380917,-2220,0,...,331,1016,0.008851,-0.043876,0.399326,15.0,12.00,11.5,12.0,10.0
2019-04-03 16:57:00,buy,9,94665,96035,93135,1090753,-0.0147,-0.794306,1926,1,...,31,65,-0.017028,-0.043876,0.399326,10.0,12.00,11.5,12.0,12.5
2019-04-03 17:00:01,buy,9,94670,96035,93135,1092670,0.0001,-0.030068,1917,1,...,151,330,0.000000,-0.043876,0.399326,15.0,12.00,11.5,14.0,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23 14:31:14,buy,38,129450,129435,126535,1338744,-0.0029,-0.182685,-2274,0,...,34,202,-0.003736,-0.043876,0.399326,5.0,13.25,11.0,11.0,7.5
2021-06-24 16:45:16,buy,38,130130,130115,127215,1336430,0.0053,0.239754,-2314,0,...,43,271,0.005682,-0.043876,0.399326,10.0,13.25,10.5,9.0,7.5
2021-06-25 16:57:00,buy,19,127720,129590,126690,1342446,-0.0185,-0.993755,6016,1,...,56,173,-0.018142,-0.043876,0.399326,10.0,13.50,11.0,8.0,10.0


In [ ]:
strat_df = pd.merge(df, concat_df, how='inner', left_index=True, right_index=True)

In [223]:
strat_df.to_csv('C:/Users/BTK/Desktop/Eric/Git/backtest_model/database/STRAT_POLENTOVISK_WIN.csv', index=True)